In [1]:
const sqlite3 = require('better-sqlite3');


In [13]:
const http = require('http');

In [2]:
db = new sqlite3('foobar.db')


Database {
  inTransaction: false,
  open: true,
  memory: false,
  readonly: false,
  name: 'foobar.db'
}

In [144]:
stmt = db.prepare('DROP TABLE foo');
info = stmt.run();

SqliteError: no such table: foo

In [146]:
sql_init = 'CREATE TABLE IF NOT EXISTS foo(_1,b);'
stmt = db.prepare(sql_init)
info = stmt.run()

{ changes: 0, lastInsertRowid: 3 }

In [4]:
stmt = db.prepare('INSERT INTO foo (a, b) VALUES (?, ?)');
info = stmt.run(1, 2);


{ changes: 1, lastInsertRowid: 1 }

In [5]:
params = [3, 4];
stmt = db.prepare('INSERT INTO foo (a, b) VALUES (?, ?)');
info = stmt.run(...params)

{ changes: 1, lastInsertRowid: 2 }

In [209]:
params = [5, 6];
stmt = db.prepare('INSERT INTO foo VALUES (8, 9)');
info = stmt.run()

{ changes: 1, lastInsertRowid: 4 }

In [190]:
temp = db.prepare('SELECT b FROM foo;')
temp.raw()
a = temp.all()
a = [].concat(...a)

[ 6, 8, null ]

In [194]:
typeof(a[2])

'object'

In [12]:
try {
    stmt = db.prepare('INSERT INTO foo VALUES (?)');
    info = stmt.run(7);
} catch (err) {
    if (err.name=='SqliteError') {
        console.log('caught error');
    }
    console.log(typeof(err))
    console.log(err);
}

caught error
object
SqliteError: table foo has 2 columns but 1 values were supplied
    at evalmachine.<anonymous>:2:15
    at Script.runInThisContext (vm.js:120:18)
    at Object.runInThisContext (vm.js:309:38)
    at run ([eval]:1054:15)
    at onRunRequest ([eval]:888:18)
    at onMessage ([eval]:848:13)
    at process.emit (events.js:315:20)
    at emit (internal/child_process.js:876:12)
    at processTicksAndRejections (internal/process/task_queues.js:85:21)


In [66]:
const url_all = 'http://localhost:5000/fridge/all'
const url_keys = 'http://localhost:5000/fridge/keys'

function get_json(url) {
    return new Promise((resolve, reject) => {
        http.get(url, res => {
            res.setEncoding('utf-8');
            let body = ''
            res.on("data", data => {body+=data;});
            res.on("end", ()=> {
                body = JSON.parse(body);
                //console.log('done');
                resolve(body)
            });
        });
    });
}



SyntaxError: Identifier 'url_all' has already been declared

In [43]:
function get_keys() {
    let keys;
    get_json(url_keys).then(res=> {
        keys=res
    });
    console.log('keys', keys)
    return keys
}


In [159]:
(async () => { 
    keys = await get_json(url_keys)
    let sql_columns = keys.map(item => { 
        item = /^\d/.test(item) ? '_'+item : item;
        item = item.replace(/ /g, '_');
        return item; 
    });
    let columns = sql_columns.join(', ');
    let table_name = 'fridge'
    let sql_create = `CREATE TABLE IF NOT EXISTS ${table_name}(${columns});`
    console.log(sql_create)
    let stmt = db.prepare(sql_create)
    let info = stmt.run();
    console.log(info);
})();

CREATE TABLE IF NOT EXISTS fridge(Time, HR_Time, _1k, switch, pcb_1, _4k, _40k, pcb_2, out1, out_1_i, out_1_v, out_1_r, pcb_3, hp, out_2_i, out_2_v, out_2_r, pcb_4, hs, bias, aio_3, pump, dio, relays, v1, v2, v3, compressor);
{ changes: 0, lastInsertRowid: 1 }


In [196]:
all = {}
get_json(url_all).then(res=>all = res)


{
  Time: '1537018190.32',
  HR_Time: 'September 15  2018 07:29:50 AM',
  '1k': '19.32311',
  switch: '295.8638',
  'pcb 1': '308',
  '4k': '295.5218',
  '40k': '294.4887',
  'pcb 2': '308',
  out1: '0.000000',
  'out 1 i': '0.000000',
  'out 1 v': '0.000000',
  'out 1 r': 'NaN',
  'pcb 3': '300.1466',
  hp: '0.000000',
  'out 2 i': '0.000000',
  'out 2 v': '0.000000',
  'out 2 r': 'NaN',
  'pcb 4': '299.7195',
  hs: '0.000000',
  bias: '1.573098',
  'aio 3': '0.999974',
  pump: '1.573113',
  dio: '0',
  relays: '0',
  v1: 'NaN',
  v2: 'NaN',
  v3: 'NaN',
  compressor: '-1'
}

In [212]:

row = Object.values(all)
row = row.map(item => {
    if (item == 'NaN') {return 'null'}
    else if (/^[0-9,-]/.test(item)) {return item}
    else {return "\'"+item+"\'"}
});
row = row.join(', ')

"1537018190.32, 'September 15  2018 07:29:50 AM', 19.32311, 295.8638, 308, 295.5218, 294.4887, 308, 0.000000, 0.000000, 0.000000, null, 300.1466, 0.000000, 0.000000, 0.000000, null, 299.7195, 0.000000, 1.573098, 0.999974, 1.573113, 0, 0, null, null, null, -1"

In [213]:
row_insert = `INSERT INTO fridge VALUES(${row});`
stmt = db.prepare(row_insert)
info = stmt.run();

{ changes: 1, lastInsertRowid: 1 }

In [214]:
temp = db.prepare('SELECT * FROM fridge;')
temp.raw()
a = temp.all()
a = [].concat(...a)

[
  1537018190.32,
  'September 15  2018 07:29:50 AM',
  19.32311,
  295.8638,
  308,
  295.5218,
  294.4887,
  308,
  0,
  0,
  0,
  null,
  300.1466,
  0,
  0,
  0,
  null,
  299.7195,
  0,
  1.573098,
  0.999974,
  1.573113,
  0,
  0,
  null,
  null,
  null,
  -1
]

In [95]:
r=[];
r = get_json(url_keys)
r.then(res=>r=res.map(item => item.replace(/ /g, '_')))

[
  'Time',       'HR_Time', '1k',
  'switch',     'pcb_1',   '4k',
  '40k',        'pcb_2',   'out1',
  'out_1_i',    'out_1_v', 'out_1_r',
  'pcb_3',      'hp',      'out_2_i',
  'out_2_v',    'out_2_r', 'pcb_4',
  'hs',         'bias',    'aio_3',
  'pump',       'dio',     'relays',
  'v1',         'v2',      'v3',
  'compressor'
]

In [97]:
r.join(', ')

'Time, HR_Time, 1k, switch, pcb_1, 4k, 40k, pcb_2, out1, out_1_i, out_1_v, out_1_r, pcb_3, hp, out_2_i, out_2_v, out_2_r, pcb_4, hs, bias, aio_3, pump, dio, relays, v1, v2, v3, compressor'

In [79]:
k.toString()

'[object Promise]'

In [70]:
r.valueOf()

[
  'Time',       'HR_Time', '1k',
  'switch',     'pcb 1',   '4k',
  '40k',        'pcb 2',   'out1',
  'out 1 i',    'out 1 v', 'out 1 r',
  'pcb 3',      'hp',      'out 2 i',
  'out 2 v',    'out 2 r', 'pcb 4',
  'hs',         'bias',    'aio 3',
  'pump',       'dio',     'relays',
  'v1',         'v2',      'v3',
  'compressor'
]

In [98]:
const fetch = require('node-fetch');


In [104]:
keys = [];
fetch(url_keys).then(res => keys = res.json())
console.log(keys)

[]


In [105]:
keys

[
  'Time',       'HR_Time', '1k',
  'switch',     'pcb 1',   '4k',
  '40k',        'pcb 2',   'out1',
  'out 1 i',    'out 1 v', 'out 1 r',
  'pcb 3',      'hp',      'out 2 i',
  'out 2 v',    'out 2 r', 'pcb 4',
  'hs',         'bias',    'aio 3',
  'pump',       'dio',     'relays',
  'v1',         'v2',      'v3',
  'compressor'
]

In [133]:
out = [];
request = async () => {
    response = await fetch(url_keys);
    json = await response.json();
    json = json.map(item=>item.replace(/ /g,'_'))
    console.log(json)
    //out = json;
}

[AsyncFunction: request]

In [134]:
request();
console.log(out)

[]
[
  'Time',       'HR_Time', '1k',
  'switch',     'pcb_1',   '4k',
  '40k',        'pcb_2',   'out1',
  'out_1_i',    'out_1_v', 'out_1_r',
  'pcb_3',      'hp',      'out_2_i',
  'out_2_v',    'out_2_r', 'pcb_4',
  'hs',         'bias',    'aio_3',
  'pump',       'dio',     'relays',
  'v1',         'v2',      'v3',
  'compressor'
]


In [132]:
out

[
  'Time',       'HR_Time', '1k',
  'switch',     'pcb_1',   '4k',
  '40k',        'pcb_2',   'out1',
  'out_1_i',    'out_1_v', 'out_1_r',
  'pcb_3',      'hp',      'out_2_i',
  'out_2_v',    'out_2_r', 'pcb_4',
  'hs',         'bias',    'aio_3',
  'pump',       'dio',     'relays',
  'v1',         'v2',      'v3',
  'compressor'
]

In [152]:
keys[2]

'1k'

In [153]:
/^\d/.test(keys[2])

true

In [157]:
keys.map(item => { item = /^\d/.test(item) ? '_'+item : item;
                  item = item.replace(/ /g, '_');
                  return item; })

[
  'Time',       'HR_Time', '_1k',
  'switch',     'pcb_1',   '_4k',
  '_40k',       'pcb_2',   'out1',
  'out_1_i',    'out_1_v', 'out_1_r',
  'pcb_3',      'hp',      'out_2_i',
  'out_2_v',    'out_2_r', 'pcb_4',
  'hs',         'bias',    'aio_3',
  'pump',       'dio',     'relays',
  'v1',         'v2',      'v3',
  'compressor'
]